In [1]:
import brightway2 as bw
import pandas as pd
import numpy as np

In [2]:
bw.projects.set_current('exiobase_reproduce_results')
co = bw.Database('CH consumption 1.0')
average_consump = co.search('ch hh average consumption')[0]
# average_consump = co.search('food')[0]

In [3]:
sectors = [act for act in co if 'sector' in act["name"]]

In [4]:
set([act for act in co if act.get("category_coarse") is None])

{'Alcoholic beverages and tobacco sector' (1 month of consumption, CH, None),
 'Clothing and footwear sector' (1 month of consumption, CH, None),
 'Communication sector' (1 month of consumption, CH, None),
 'Durable goods sector' (1 month of consumption, CH, None),
 'Education sector' (1 month of consumption, CH, None),
 'Fees sector' (1 month of consumption, CH, None),
 'Food and non-alcoholic beverages sector' (1 month of consumption, CH, None),
 'Furnishings, household equipment and routine household maintenance sector' (1 month of consumption, CH, None),
 'Health sector' (1 month of consumption, CH, None),
 'Housing, water, electricity, gas and other fuels sector' (1 month of consumption, CH, None),
 'Miscellaneous goods and services sector' (1 month of consumption, CH, None),
 'Other insurance premiums sector' (1 month of consumption, CH, None),
 'Premiums for life insurance sector' (1 month of consumption, CH, None),
 'Recreation and culture sector' (1 month of consumption, CH, N

In [5]:
[act for act in co if "Desktop computers" in act['name']][0].as_dict()

{'reference product': 'Desktop computers',
 'code': 'cg_nodesktoppcs',
 'location': 'CH',
 'amount': 1.0,
 'unit': 'unit',
 'name': 'Desktop computers',
 'worksheet name': 'Sheet1',
 'database': 'CH consumption 1.0',
 'category_coarse': 'Durable goods'}

In [ ]:
# list(average_consump.exchanges())

In [6]:
# list_functional_units = []
# for exc in average_consump.exchanges():
#     if exc['type'] != 'production':
#         list_functional_units.append({exc['input']: exc['amount']})

list_functional_units = []
for act in sectors:
    list_functional_units.append({act: 1})
        
list_methods = [('IPCC 2013', 'climate change', 'GTP 100a')]

In [7]:
bw.calculation_setups['av_consump_contribution_analysis'] = {'inv':list_functional_units, 'ia':list_methods}

In [8]:
%%time
myMultiLCA = bw.MultiLCA('av_consump_contribution_analysis')

CPU times: user 1.89 s, sys: 167 ms, total: 2.05 s
Wall time: 918 ms


In [9]:
lcia_unit = bw.Method(list_methods[0]).metadata['unit']

In [10]:
fus = [bw.get_activity(list(el.keys())[0])['name'][:25] for el in list_functional_units]
df = pd.DataFrame(index=fus, columns=[lcia_unit], data=myMultiLCA.results)
df['exchanges'] = [list(fu.values())[0] for fu in list_functional_units]
df['units'] = [bw.get_activity(list(el.keys())[0])['unit'] for el in list_functional_units]


In [11]:
df.sort_values(lcia_unit, ascending=False, inplace=True)

In [12]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df)

                            kg CO2-Eq  exchanges                   units
Transport sector           409.979029          1  1 month of consumption
Housing, water, electrici  389.615730          1  1 month of consumption
Food and non-alcoholic be  206.095185          1  1 month of consumption
Recreation and culture se  153.272810          1  1 month of consumption
Furnishings, household eq  121.576936          1  1 month of consumption
Durable goods sector       110.239413          1  1 month of consumption
Restaurants and hotels se   87.185604          1  1 month of consumption
Miscellaneous goods and s   71.495445          1  1 month of consumption
Clothing and footwear sec   57.119692          1  1 month of consumption
Health sector               55.609832          1  1 month of consumption
Communication sector        23.933041          1  1 month of consumption
Premiums for life insuran   13.478634          1  1 month of consumption
Other insurance premiums     9.093726          1  1

In [13]:
df["kg CO2-Eq"].values.sum()

1725.9909161295304

In [ ]:
# df_ = df.sort_values(lcia_unit)
# df_['kg CO2-Eq'].plot.barh(figsize=(20,40))

# Biosphere

In [ ]:
import bw2analyzer

In [ ]:
lca = bw.LCA({average_consump: 1}, list_methods[0])

In [ ]:
lca.lci()
lca.lcia()
lca.score

In [ ]:
ca = bw2analyzer.ContributionAnalysis()

In [ ]:
ca.annotated_top_processes(lca)

# Contribution analysis for sectors

In [ ]:
list_methods = [('IPCC 2013', 'climate change', 'GTP 100a')]
list_fus = []
for cat_of_interest in category_activities:
    list_fus.append({co.search(cat_of_interest)[0]: 1})
    
bw.calculation_setups['sector_contribution_analysis'] = {'inv':list_fus, 'ia':list_methods}

%%time
myMultiLCA = bw.MultiLCA('sector_contribution_analysis')

lcia_unit = bw.Method(list_methods[0]).metadata['unit']

fus = [bw.get_activity(list(el.keys())[0])['name'][:25] for el in list_fus]
df = pd.DataFrame(index=fus, columns=[lcia_unit], data=myMultiLCA.results)
df['units'] = [bw.get_activity(list(el.keys())[0])['unit'] for el in list_fus]

df.sort_values(lcia_unit, ascending=False, inplace=True)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df)